# Facebook-notify: get emailed new facebook group posts immmediatelly 


### Step 1: Configure

In [1]:
facebook_username = ''
facebook_password = ''
facebook_group_id = ''
gmail_username = ''
gmail_password = ''
gmail_reciever = ''
refresh_frequency = 10 #in minutes, if its too low, you'll get banned

### Step 2: define

In [2]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from pyvirtualdisplay import Display
from PIL import Image
import time
import smtplib
from io import BytesIO
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

In [3]:
#helper function to display the current state of the browser
def screenshot_browser():
    png = driver.get_screenshot_as_png()
    return Image.open(BytesIO(png)) 

#allows creating a browser window in a headless environment
def create_virtual_display():
    display = Display(visible=0, size=(1080, 1920))
    display.start()

def init_browser():
    # configure browser to disable notification popups
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    global driver
    #initialize browser window
    driver = webdriver.Chrome(executable_path = '/usr/lib/chromium-browser/chromedriver',chrome_options = chrome_options)
        
def emailImage(subject, image, link = ''):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = gmail_username
    msg['To'] = gmail_reciever
    
    text = MIMEText('<a href="{}"><img src="cid:image1"></a>'.format(link), 'html')
    msg.attach(text)

    image = MIMEImage(image)

    # Define the image's ID as referenced in the HTML body above
    image.add_header('Content-ID', '<image1>')
    msg.attach(image)

    try:
        smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        smtp_server.ehlo()
        smtp_server.login(gmail_username, gmail_password)
        smtp_server.sendmail(gmail_username, [gmail_reciever],  msg.as_string())
        smtp_server.close()
        print ("Email sent successfully!")
    except Exception as ex:
        print ("Email not sent",ex)
            
def facebook_login():
    #navigate to facebook.com
    driver.get("http://www.facebook.com")

    #accept cookies
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@data-testid="cookie-policy-dialog-accept-button"]'))).click()

    #get username & password fields
    username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
    password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

    #enter username and password
    username_field.clear()
    username_field.send_keys(facebook_username)
    password_field.clear()
    password_field.send_keys(facebook_password)

    #target the login button and click it
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    #wait a bit for good meassure
    time.sleep(10)

def update_previous_name(name):
    global previous_name
    previous_name = name
    f = open("previous_name.txt", "w")
    f.write(name)
    f.close()
    
def load_previous_name():
    global previous_name
    try:
        f = open("previous_name.txt", "r")
        previous_name = f.read()
        f.close()
    except:
        previous_name = ''
    
def email_new_post():
    driver.get("http://www.facebook.com/groups/{}/?sorting_setting=CHRONOLOGICAL".format(facebook_group_id))
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@role='feed']/div[2]//strong[1]/span"))
    )
    
    global previous_name
    current_name = driver.find_element(By.XPATH, "//div[@role='feed']/div[2]//strong[1]/span").text
    if previous_name == current_name: return
    print('new post from ' + current_name, end = ". ")
    
    link_element = driver.find_elements(By.XPATH,"//div[@role='feed']/div[2]//a")[3]
    ActionChains(driver).move_to_element(link_element).perform()
    link = link_element.get_attribute('href')
    
    listing_image = driver.find_element(By.XPATH, "//div[@role='feed']/div[2]").screenshot_as_png    
    
    emailImage("NEW LISTING", listing_image, link)
    
    update_previous_name(current_name)
    
    
def run_loop():
    while True:
        try:
            email_new_post()
        except Exception as ex:
            print ("Something went wrong….",ex)
        time.sleep(refresh_frequency*60)

### Step 3: run

In [4]:
# execute setup steps
create_virtual_display()
init_browser()
facebook_login()
load_previous_name()

/tmp/ipykernel_26433/1682249705.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = '/usr/lib/chromium-browser/chromedriver',chrome_options = chrome_options)
/tmp/ipykernel_26433/1682249705.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = '/usr/lib/chromium-browser/chromedriver',chrome_options = chrome_options)


In [ ]:
#run infinite loop
run_loop()

new post from Andreas Anker Seelhorst Rasmussen. Email sent successfully!


In [ ]:
screenshot_browser()